# Credit Score Project

---

In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle, islice

In [13]:
# Display dataset with no index column
raw_df = pd.read_csv('train_biased.csv')
df = raw_df.copy()
raw_df

,ID,Customer_ID,Month,Name,City,Street,Age,SSN,Occupation,Annual_Income,...,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Credit_Score
0,NaN,CUS_0xd40,NaN,Aaron Maashoh,Lonton,Oxford Street,NaN,821-00-0265,Manager,19114.12,...,7,11.27,4.0,_,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253,3
1,0x1603,CUS_0xd40,February,Aaron Maashoh,Lonton,Oxford Street,23.0,821-00-0265,Manager,19114.12,...,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736,3
2,0x1604,CUS_0xd40,NaN,NaN,Lonton,Oxford Street,-500.0,821-00-0265,Manager,19114.12,...,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648,3
3,0x1605,CUS_0xd40,April,Aaron Maashoh,Lonton,Oxford Street,23.0,821-00-0265,Manager,19114.12,...,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713,3
4,NaN,CUS_0xd40,May,Aaron Maashoh,Lonton,Oxford Street,23.0,821-00-0265,Manager,19114.12,...,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,BadShire,North Street,NaN,078-73-5990,Cleaner,39628.99,...,7,11.5,NaN,_,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485,1
99996,0x25fea,CUS_0x942c,May,Nicks,BadShire,North Street,25.0,078-73-5990,Cleaner,39628.99,...,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385,1
99997,0x25feb,NaN,June,Nicks,BadShire,North Street,25.0,078-73-5990,Cleaner,NaN,...,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441,1
99998,0x25fec,CUS_0x942c,July,Nicks,Standhampton,North Street,25.0,078-73-5990,Cleaner,39628.99,...,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603,2


In [14]:
# Create a copy of the 'Credit_Score' column
credit_score_df = raw_df['Credit_Score'].copy()
credit_score_df

0        3
1        3
2        3
3        3
4        3
        ..
99995    1
99996    1
99997    1
99998    2
99999    1
Name: Credit_Score, Length: 100000, dtype: int64

In [15]:
# Remove the 'ID', 'SSN' and 'Name' column from the DataFrame
df.drop(columns=['ID', 'SSN', 'Name'], inplace=True, axis=1)
df

,Customer_ID,Month,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Credit_Score
0,CUS_0xd40,NaN,Lonton,Oxford Street,NaN,Manager,19114.12,1824.843333,3.0,4.0,...,7,11.27,4.0,_,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253,3
1,CUS_0xd40,February,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,4.0,...,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736,3
2,CUS_0xd40,NaN,Lonton,Oxford Street,-500.0,Manager,19114.12,NaN,3.0,NaN,...,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648,3
3,CUS_0xd40,April,Lonton,Oxford Street,23.0,Manager,19114.12,NaN,3.0,4.0,...,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713,3
4,CUS_0xd40,May,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS_0x942c,April,BadShire,North Street,NaN,Cleaner,39628.99,3359.415833,4.0,6.0,...,7,11.5,NaN,_,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485,1
99996,CUS_0x942c,May,BadShire,North Street,25.0,Cleaner,39628.99,NaN,4.0,6.0,...,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385,1
99997,NaN,June,BadShire,North Street,25.0,Cleaner,NaN,NaN,4.0,6.0,...,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441,1
99998,CUS_0x942c,July,Standhampton,North Street,25.0,Cleaner,39628.99,NaN,NaN,6.0,...,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603,2


In [16]:
# Defining the columns to be processed
columns_to_process = [
    'Customer_ID', 'City', 'Street', 'Age', 
    'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_of_Loan'
]

# Process the DataFrame in chunks of 8 rows
chunk_size = 8
for start in range(0, len(df), chunk_size):
    end = start + chunk_size
    # Use indices to work with the chunk
    chunk_indices = df.index[start:end]
    chunk = df.loc[chunk_indices]
    # For each specified column, calculate the mode and update the chunk rows
    for col in columns_to_process:
        # If there are multiple modes, we take the first one
        mode_value = chunk[col].mode().iloc[0]
        df.loc[chunk_indices, col] = mode_value

df

,Customer_ID,Month,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Credit_Score
0,CUS_0xd40,NaN,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,7,11.27,4.0,_,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253,3
1,CUS_0xd40,February,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736,3
2,CUS_0xd40,NaN,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,NaN,...,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648,3
3,CUS_0xd40,April,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713,3
4,CUS_0xd40,May,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS_0x942c,April,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,6.0,...,7,11.5,NaN,_,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485,1
99996,CUS_0x942c,May,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,6.0,...,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385,1
99997,CUS_0x942c,June,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,6.0,...,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441,1
99998,CUS_0x942c,July,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,NaN,6.0,...,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603,2


In [17]:
# Define month encoding
month_to_number = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8
}

# Encode the known months
df["Month"] = df["Month"].map(month_to_number)

# Fill in the missing values using the repeating sequence 1-8
encoded_cycle = list(islice(cycle(range(1, 9)), len(df)))
df["Month"] = encoded_cycle
df

,Customer_ID,Month,City,Street,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Credit_Score
0,CUS_0xd40,1,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,7,11.27,4.0,_,NaN,26.822620,22 Years and 1 Months,No,80.41529543900253,3
1,CUS_0xd40,2,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,NaN,11.27,4.0,NaN,809.98,31.944960,NaN,No,118.28022162236736,3
2,CUS_0xd40,3,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,NaN,...,7,NaN,4.0,Good,809.98,28.609352,NaN,No,81.699521264648,3
3,CUS_0xd40,4,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,4,6.27,4.0,Good,809.98,NaN,22 Years and 4 Months,No,199.4580743910713,3
4,CUS_0xd40,5,Lonton,Oxford Street,23.0,Manager,19114.12,1824.843333,3.0,4.0,...,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,41.420153086217326,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS_0x942c,4,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,6.0,...,7,11.5,NaN,_,502.38,34.663572,31 Years and 6 Months,No,60.97133255718485,1
99996,CUS_0x942c,5,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,6.0,...,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,54.18595028760385,1
99997,CUS_0x942c,6,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,4.0,6.0,...,6,11.5,NaN,Good,502.38,41.255522,31 Years and 8 Months,No,24.02847744864441,1
99998,CUS_0x942c,7,BadShire,North Street,25.0,Cleaner,39628.99,3359.415833,NaN,6.0,...,NaN,11.5,3.0,Good,NaN,33.638208,31 Years and 9 Months,No,251.67258219721603,2
